<a href="https://colab.research.google.com/github/Ahmedzayan1/Netowrk-anomaly-detection/blob/main/Assignment2_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2
Omar Ahmed ElSammak 6867<br>
Aly Hamdy Ibrahim   6760<br>
Ahmed Mohamed Zayan 6714

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from numpy.random import uniform
from sklearn.datasets import make_blobs
import seaborn as sns
import random
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from numpy.core.fromnumeric import argmax
import math
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/KDD/kddcup.data_10_percent_corrected'
path2 = '/content/drive/MyDrive/KDD/corrected'
path3 = '/content/drive/MyDrive/KDD/kddcup.data.corrected'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Data reading and encoding**

In [ ]:
cols="""duration,protocol_type,
service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,
root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,
is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,
dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,
dst_host_rerror_rate,dst_host_srv_rerror_rate"""

columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())

columns.append('target')
#print(columns)
print(len(columns))

42


In [ ]:
df = pd.read_csv(path,names=columns)
df2= pd.read_csv(path2,names=columns)
df3= pd.read_csv(path3,names=columns)

In [ ]:
df2.shape

(311029, 42)

In [ ]:
df3.shape

(4898431, 42)

In [ ]:
print(df['target'].nunique())
print(df2['target'].nunique())
print(df3['target'].nunique())

23
38
23


In [ ]:
print(df['target'].unique())
print(df2['target'].unique())

['normal.' 'buffer_overflow.' 'loadmodule.' 'perl.' 'neptune.' 'smurf.'
 'guess_passwd.' 'pod.' 'teardrop.' 'portsweep.' 'ipsweep.' 'land.'
 'ftp_write.' 'back.' 'imap.' 'satan.' 'phf.' 'nmap.' 'multihop.'
 'warezmaster.' 'warezclient.' 'spy.' 'rootkit.']
['normal.' 'snmpgetattack.' 'named.' 'xlock.' 'smurf.' 'ipsweep.'
 'multihop.' 'xsnoop.' 'sendmail.' 'guess_passwd.' 'saint.'
 'buffer_overflow.' 'portsweep.' 'pod.' 'apache2.' 'phf.' 'udpstorm.'
 'warezmaster.' 'perl.' 'satan.' 'xterm.' 'mscan.' 'processtable.' 'ps.'
 'nmap.' 'rootkit.' 'neptune.' 'loadmodule.' 'imap.' 'back.' 'httptunnel.'
 'worm.' 'mailbomb.' 'ftp_write.' 'teardrop.' 'land.' 'sqlattack.'
 'snmpguess.']


In [ ]:
diff_targets=['snmpgetattack.', 'named.', 'xlock.'
       ,'xsnoop.', 'sendmail.',
       'saint.','apache2.',
        'udpstorm.', 'xterm.',
       'mscan.', 'processtable.', 'ps.',
       'httptunnel.', 'worm.',
       'mailbomb.', 'sqlattack.',
       'snmpguess.','warezclient.','spy.']
for target in diff_targets:
  df2.drop(df2[df2['target'] == target].index, inplace = True)

In [ ]:
print(df['target'].nunique())
print(df2['target'].nunique())

23
21


In [ ]:
diff_targets=['warezclient.','spy.']
for target in diff_targets:
  df.drop(df[df['target'] == target].index, inplace = True)

In [ ]:
print(df['target'].nunique())
print(df2['target'].nunique())

21
21


In [ ]:
print(df['target'].unique())
print(df2['target'].unique())

['normal.' 'buffer_overflow.' 'loadmodule.' 'perl.' 'neptune.' 'smurf.'
 'guess_passwd.' 'pod.' 'teardrop.' 'portsweep.' 'ipsweep.' 'land.'
 'ftp_write.' 'back.' 'imap.' 'satan.' 'phf.' 'nmap.' 'multihop.'
 'warezmaster.' 'rootkit.']
['normal.' 'smurf.' 'ipsweep.' 'multihop.' 'guess_passwd.'
 'buffer_overflow.' 'portsweep.' 'pod.' 'phf.' 'warezmaster.' 'perl.'
 'satan.' 'nmap.' 'rootkit.' 'neptune.' 'loadmodule.' 'imap.' 'back.'
 'ftp_write.' 'teardrop.' 'land.']


In [ ]:
pd.set_option('display.max_columns', None)
df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
pd.set_option('display.max_columns', None)
df2.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
6,0,udp,domain_u,SF,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.0,0.0,0.5,1.0,0.0,10,3,0.30,0.30,0.3,0.0,0.0,0.0,0.0,0.0,normal.
7,0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,253,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.


we have four features which are object we need to change it to be numerical

In [ ]:
le = LabelEncoder()

encoding protocol type

In [ ]:
df['protocol_type'].unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

In [ ]:
df2['protocol_type'].unique()

array(['udp', 'tcp', 'icmp'], dtype=object)

In [ ]:
df['protocol_type'] = le.fit_transform(df['protocol_type'])
df2['protocol_type'] = le.fit_transform(df2['protocol_type'])
df3['protocol_type'] = le.fit_transform(df3['protocol_type'])

In [ ]:
df['protocol_type'].unique()

array([1, 2, 0])

In [ ]:
df2['protocol_type'].unique()

array([2, 1, 0])

encoding service

In [ ]:
df['service'] = le.fit_transform(df['service'])

In [ ]:
df2['service'] = le.fit_transform(df2['service'])

In [ ]:
df3['service'] = le.fit_transform(df3['service'])

In [ ]:
df['service'].nunique()

66

In [ ]:
df['service'].unique()

array([22, 50, 17, 11,  3, 56, 18, 13, 39, 14, 40, 45, 43, 19, 48, 59, 31,
       29, 47, 20, 52, 32, 65, 10, 30, 24,  8,  7, 38, 49,  0, 37, 23, 16,
       44, 15,  5, 62, 26, 27, 12,  9, 55, 54, 25, 21,  6, 42, 53, 63, 34,
       35, 33, 51, 64,  4,  2, 28, 36, 60,  1, 61, 41, 57, 58, 46])

encoding flag

In [ ]:
df['flag'].nunique()

11

In [ ]:
df['flag'] = le.fit_transform(df['flag'])
df2['flag'] = le.fit_transform(df2['flag'])
df3['flag'] = le.fit_transform(df3['flag'])


In [ ]:
df['flag'].unique()

array([ 9,  6,  1,  7,  5,  8,  2,  4,  3,  0, 10])

In [ ]:
df['flag'].nunique()

11

In [ ]:
df.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,1,22,9,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.00,0.0,0.0,0.0,0.0,normal.
1,0,1,22,9,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,normal.
2,0,1,22,9,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,normal.
3,0,1,22,9,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,normal.
4,0,1,22,9,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,normal.
5,0,1,22,9,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,59,59,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,normal.
6,0,1,22,9,212,1940,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,69,1.0,0.0,1.00,0.04,0.0,0.0,0.0,0.0,normal.
7,0,1,22,9,159,4087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,79,1.0,0.0,0.09,0.04,0.0,0.0,0.0,0.0,normal.
8,0,1,22,9,210,151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,89,1.0,0.0,0.12,0.04,0.0,0.0,0.0,0.0,normal.
9,0,1,22,9,212,786,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,99,1.0,0.0,0.12,0.05,0.0,0.0,0.0,0.0,normal.


In [ ]:
df2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
1,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
2,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
6,0,2,11,9,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.0,0.0,0.5,1.0,0.0,10,3,0.30,0.30,0.3,0.0,0.0,0.0,0.0,0.0,normal.
7,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,253,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492999 entries, 0 to 494020
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     492999 non-null  int64  
 1   protocol_type                492999 non-null  int64  
 2   service                      492999 non-null  int64  
 3   flag                         492999 non-null  int64  
 4   src_bytes                    492999 non-null  int64  
 5   dst_bytes                    492999 non-null  int64  
 6   land                         492999 non-null  int64  
 7   wrong_fragment               492999 non-null  int64  
 8   urgent                       492999 non-null  int64  
 9   hot                          492999 non-null  int64  
 10  num_failed_logins            492999 non-null  int64  
 11  logged_in                    492999 non-null  int64  
 12  num_compromised              492999 non-null  int64  
 13 

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df2.drop_duplicates(inplace=True)

In [ ]:
df3.drop_duplicates(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144691 entries, 0 to 494020
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     144691 non-null  int64  
 1   protocol_type                144691 non-null  int64  
 2   service                      144691 non-null  int64  
 3   flag                         144691 non-null  int64  
 4   src_bytes                    144691 non-null  int64  
 5   dst_bytes                    144691 non-null  int64  
 6   land                         144691 non-null  int64  
 7   wrong_fragment               144691 non-null  int64  
 8   urgent                       144691 non-null  int64  
 9   hot                          144691 non-null  int64  
 10  num_failed_logins            144691 non-null  int64  
 11  logged_in                    144691 non-null  int64  
 12  num_compromised              144691 non-null  int64  
 13 

In [ ]:
df.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,144691.000000,144691.000000,144691.000000,144691.000000,1.446910e+05,1.446910e+05,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.0,144691.0,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000
mean,128.500287,1.068152,32.234258,6.973440,5.929098e+03,2.872382e+03,0.000138,0.020326,0.000048,0.044052,0.000518,0.488351,0.026339,0.000380,0.000117,0.038758,0.003691,0.000366,0.003428,0.0,0.0,0.002612,74.837778,13.083799,0.293164,0.293336,0.109008,0.109068,0.653084,0.040781,0.093576,182.003532,130.465060,0.552296,0.061637,0.089577,0.018587,0.293321,0.292750,0.111118,0.108338
std,1213.202779,0.311071,13.035197,2.711534,1.823055e+06,6.099730e+04,0.011756,0.240102,0.010182,0.725422,0.028674,0.499866,3.321598,0.019493,0.014157,3.718941,0.178110,0.020190,0.067246,0.0,0.0,0.051046,100.480409,30.817173,0.453713,0.454380,0.309779,0.309623,0.447354,0.120556,0.242468,98.760708,114.840458,0.456582,0.147379,0.235386,0.059768,0.453336,0.454277,0.306924,0.305042
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,22.000000,5.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,79.000000,11.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,22.000000,9.000000,1.470000e+02,1.050000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,12.000000,8.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,255.000000,121.000000,0.760000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,45.000000,9.000000,2.860000e+02,1.159000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,133.000000,15.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.060000,0.000000,255.000000,255.000000,1.000000,0.070000,0.030000,0.020000,1.000000,1.000000,0.000000,0.000000
max,58329.000000,2.000000,65.000000,10.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,1.000000,2.000000,993.000000,28.000000,2.000000,8.000000,0.0,0.0,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


num_outbound_cmds and is_host_login have variance and min and max values = 0

In [ ]:
df['num_outbound_cmds'].unique()

array([0])

In [ ]:
df['is_host_login'].unique()

array([0])

drop these 2 features 

In [ ]:
# df.drop(['num_outbound_cmds'], axis=1,inplace=True)
# df.drop(['is_host_login'], axis=1,inplace=True)

In [ ]:
df.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,144691.000000,144691.000000,144691.000000,144691.000000,1.446910e+05,1.446910e+05,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.0,144691.0,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000,144691.000000
mean,128.500287,1.068152,32.234258,6.973440,5.929098e+03,2.872382e+03,0.000138,0.020326,0.000048,0.044052,0.000518,0.488351,0.026339,0.000380,0.000117,0.038758,0.003691,0.000366,0.003428,0.0,0.0,0.002612,74.837778,13.083799,0.293164,0.293336,0.109008,0.109068,0.653084,0.040781,0.093576,182.003532,130.465060,0.552296,0.061637,0.089577,0.018587,0.293321,0.292750,0.111118,0.108338
std,1213.202779,0.311071,13.035197,2.711534,1.823055e+06,6.099730e+04,0.011756,0.240102,0.010182,0.725422,0.028674,0.499866,3.321598,0.019493,0.014157,3.718941,0.178110,0.020190,0.067246,0.0,0.0,0.051046,100.480409,30.817173,0.453713,0.454380,0.309779,0.309623,0.447354,0.120556,0.242468,98.760708,114.840458,0.456582,0.147379,0.235386,0.059768,0.453336,0.454277,0.306924,0.305042
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,22.000000,5.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,79.000000,11.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,22.000000,9.000000,1.470000e+02,1.050000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,12.000000,8.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,255.000000,121.000000,0.760000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,45.000000,9.000000,2.860000e+02,1.159000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,133.000000,15.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.060000,0.000000,255.000000,255.000000,1.000000,0.070000,0.030000,0.020000,1.000000,1.000000,0.000000,0.000000
max,58329.000000,2.000000,65.000000,10.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,1.000000,2.000000,993.000000,28.000000,2.000000,8.000000,0.0,0.0,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


constructing data for clustering


In [ ]:
df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'target'],
      dtype='object')

In [ ]:
df2.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'target'],
      dtype='object')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73269 entries, 0 to 310931
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     73269 non-null  int64  
 1   protocol_type                73269 non-null  int64  
 2   service                      73269 non-null  int64  
 3   flag                         73269 non-null  int64  
 4   src_bytes                    73269 non-null  int64  
 5   dst_bytes                    73269 non-null  int64  
 6   land                         73269 non-null  int64  
 7   wrong_fragment               73269 non-null  int64  
 8   urgent                       73269 non-null  int64  
 9   hot                          73269 non-null  int64  
 10  num_failed_logins            73269 non-null  int64  
 11  logged_in                    73269 non-null  int64  
 12  num_compromised              73269 non-null  int64  
 13  root_shell     

In [ ]:
features=df.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
clustering_data=df.dropna(subset=features)
clustering_data=clustering_data[features].copy()
testing_data=df2.dropna(subset=features)
testing_data=testing_data[features].copy()

In [ ]:
clustering_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,22,9,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,1,22,9,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,1,22,9,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,1,22,9,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,1,22,9,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [ ]:
testing_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.0,0.0,0.0
6,0,2,11,9,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.0,0.0,0.5,1.0,0.00,10,3,0.30,0.30,0.30,0.00,0.00,0.0,0.0,0.0
7,0,2,45,9,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,255,253,0.99,0.01,0.00,0.00,0.00,0.0,0.0,0.0
9,0,1,22,9,223,185,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.00,71,255,1.00,0.00,0.01,0.01,0.00,0.0,0.0,0.0
11,0,1,22,9,230,260,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,3,255,1.00,0.00,0.33,0.07,0.33,0.0,0.0,0.0


In [ ]:
# create the scaler and fit it to the data
scaler = MinMaxScaler()
clustering_data_scaled = scaler.fit_transform(clustering_data)

In [ ]:
testing_data_scaled = scaler.fit_transform(testing_data)

--scale data <br>
1.initialize random centroids <br>
2.label each data point <br>
3.update centroids<br>
4.repeat 3 and 4 till centroids don't change 

### **Kmeans**

In [ ]:
numpy_clustering=clustering_data.to_numpy()


In [ ]:
clustering_data_scaleddf = pd.DataFrame(clustering_data_scaled,columns=clustering_data.columns, index=clustering_data.index)

In [ ]:
def euclideann(point, data):
    return np.sqrt(np.sum((point - data)**2, axis=1))
def euclidean(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

In [ ]:
class KMeans_implementation:
    def __init__(self, n_clusters=8, max_iter=300):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
    def fit(self, X_train) -> None:
      assert self.n_clusters <= len(X_train), #number of clusters can not be greater than number of samples
      #initialize the centroids using the kmeans++ approach 
      self.centroids = [X_train.iloc[np.random.choice(range(len(X_train)), size=1)[0], :]] #choose a random row as a centorid 
      for i in range(self.n_clusters-1):#loop through no of clusters -1
        """choosing the subsequent centroids from the remaining data points 
        based on a probability proportional to the square
        d distance away from a given point's nearest existing centroid'"""
        dists = np.sum([euclideann(X_train, centroid) for centroid in self.centroids], axis=0) 
        dists /= np.sum(dists) 
        new_centroid_ind = np.random.choice(range(len(X_train)), size=1, p=dists)[0] #get index of new centroid
        self.centroids += [X_train.iloc[new_centroid_ind,:]] #add the centroid to the list of centroids 
      self.centroids = np.array(self.centroids) #convert to a numpy array
      
      prev_centroids = None
      iterations = 0

      #Main kmeans loop 
      for iteration in tqdm(range(self.max_iter)):#loop till we reach max iters or when we converge
        if np.equal(self.centroids, prev_centroids).all(): # if centroids are same as previous break 
          break
        points = [[] for i in range(self.n_clusters)]#initializes an empty list points to hold the data points assigned to each cluster
        self.labels = []#initializes an empty list labels to hold the cluster assignments for each data point.
        for i in range(len(X_train)):#loop over all rows in data
          x = X_train.iloc[i, :] #choose a row
          dists = euclideann([x], self.centroids) #calculate all eculidean dist for this point with centroids 
          label = np.argmin(dists) #choose the index of the min dist 
          points[label].append(x) # append  the row to the index of the closest centroid 
          self.labels.append(label)#add the cluster assignment 

        prev_centroids = self.centroids#save the current set of cluster centroids as prev_centroids.
        self.centroids = np.array([np.mean(cluster, axis=0) for cluster in points])#get the new centorid of each cluster as the mean of it 
        for i, centroid in enumerate(self.centroids):
          if np.isnan(centroid).any():
            self.centroids[i] = prev_centroids[i]
    

def _create_clusters(kmean, centroids,X_test): 
    # assign the samples of testing data  to the closest centroids
    clusters = [[] for _ in range(kmean.n_clusters)]#create list of empty sublists of number of clusters  
    for idx, sample in enumerate(X_test):#loop through test data
        distances = [euclidean(sample, point) for point in centroids]#calculate the Euclidean distance between the sample and each of the centroids
        closest_idx = np.argmin(distances) #find the index of the closest centroid to that sample
        closest_idx = distances.index(min(distances))
        clusters[closest_idx].append(idx)#appends the index of the current sample to the corresponding cluster
    return clusters # each sublist contains the indices of the samples that belong to a particular cluster



def classify(clusters):
  clusterdf = pd.DataFrame(columns =df2['target'].unique())
  clusterdf = pd.DataFrame(0, index=np.arange(len(clusters)), columns=df2['target'].unique())#new df where columns are target
  for cluster_idx, cluster in enumerate(clusters):# loop over each cluster
    for sample_idx in cluster:#loop over each index of the sample in that cluster
      lab=df2['target'].iloc[sample_idx] #get the sample real label  using its index
      clusterdf.at[cluster_idx,lab]=clusterdf.at[cluster_idx,lab]+1 # increment the count for this cluster index at this label
  return clusterdf 

In [ ]:
occur=df2['target'].value_counts()
occurdf=occur.to_frame().transpose()

In [ ]:
def get_conditional_entropy_per_cluster(lst):
  entropy=0
  for i in lst:
    if i==0:
      continue
    else:
      ent=(-i/lst.sum())*math.log(i/lst.sum(),2)
      entropy+=ent
  return entropy

In [ ]:
def evaluate(cluster):
  col=cluster.columns.to_numpy()
  # convert columns to numpy array to get label of indexes of each maximum element.
  clusarr=cluster.to_numpy()
  f1=0
  e=0
  p=0
  r=0

  for cluster, classcount in enumerate(clusarr): #get measures for each cluster 
    maxindex=argmax(classcount) # maximum index number
    maxnumber=classcount[maxindex]
    class_count=0
    clus_arr=0
    clus_arr=clusarr.sum()
    class_count=classcount.sum()
    if maxnumber==0: #maximum element in cluster is 0 so no elements in this cluster 
      print(f"cluster {cluster+1} is empty")
      print(f"----------------")
      continue
    maxclass=col[maxindex] #maximum class column name 
    print(f"label of cluster number  {cluster+1}-> {maxclass}")
    #calculate the purity
    purity=maxnumber/classcount.sum()
    #calculate recall
    recall=maxnumber/occurdf[maxclass][0]
    f1score=(2*purity*recall)/(purity + recall)
    entro_per_cluster=get_conditional_entropy_per_cluster(classcount)
    print("within cluster measures:")
    print(f"purity= {purity}")
    print(f"recall= {recall}") 
    print(f"f1 score= {f1score}")
    print(f"----------------")
    ratio=class_count/clus_arr
    p+=purity*(ratio)
    r+=recall*(ratio)
    f1+=f1score*(ratio)
    e+=entro_per_cluster*(ratio)
  f1=(2*p*r)/(p+r)
  print("External evaluation:")
  print(f"total purity= {p}")
  print(f"total recall= {r}")
  print(f"total f1 score= {f1}")
  print(f"conditional entropy= {e}")

### **K=7**

In [ ]:
centers = 7
k7 = KMeans_implementation(n_clusters=centers)
k7.fit(clustering_data_scaleddf)
centroids7=k7.centroids
clusters7=_create_clusters(k7,centroids7,testing_data_scaled)

  7%|▋         | 21/300 [07:00<1:33:12, 20.05s/it]


In [ ]:
k7c=classify(clusters7)

In [ ]:
evaluate(k7c)

label of cluster number  1-> normal.
within cluster measures:
purity= 0.9684243032502123
recall= 0.5236157201594557
f1 score= 0.6797166041263089
----------------
label of cluster number  2-> neptune.
within cluster measures:
purity= 0.5536480686695279
recall= 0.00634467833956325
f1 score= 0.012545587162654998
----------------
label of cluster number  3-> neptune.
within cluster measures:
purity= 0.9520138888888889
recall= 0.6742573283494
f1 score= 0.7894161004261201
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 0.8699031356483786
recall= 0.4742136789597813
f1 score= 0.6138156472876595
----------------
label of cluster number  5-> neptune.
within cluster measures:
purity= 0.975609756097561
recall= 0.14164863269722605
f1 score= 0.2473801752276241
----------------
label of cluster number  6-> neptune.
within cluster measures:
purity= 1.0
recall= 0.17770017706079086
f1 score= 0.30177490081436625
----------------
label of cluster number  7-> normal.


### **K=15**

In [ ]:
centers = 15
k15 = KMeans_implementation(n_clusters=centers)
k15.fit(clustering_data_scaleddf)
centroids15=k15.centroids
clusters15=_create_clusters(k15,centroids15,testing_data_scaled)

  9%|▊         | 26/300 [07:07<1:15:06, 16.45s/it]


In [ ]:
k15c=classify(clusters15)
evaluate(k15c)

label of cluster number  1-> neptune.
within cluster measures:
purity= 1.0
recall= 0.021099744245524295
f1 score= 0.04132748904195366
----------------
label of cluster number  2-> normal.
within cluster measures:
purity= 0.9093122337188071
recall= 0.06236303299730762
f1 score= 0.11672102970761147
----------------
label of cluster number  3-> neptune.
within cluster measures:
purity= 0.9520138888888889
recall= 0.6742573283494
f1 score= 0.7894161004261201
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 1.0
recall= 0.4079059962849331
f1 score= 0.5794506129830855
----------------
label of cluster number  5-> smurf.
within cluster measures:
purity= 0.40487804878048783
recall= 0.9754273504273504
f1 score= 0.572234409276089
----------------
label of cluster number  6-> normal.
within cluster measures:
purity= 0.9500656280317298
recall= 0.34746311022060816
f1 score= 0.5088330582553945
----------------
label of cluster number  7-> neptune.
within cluster m

### **K=23**

In [ ]:
centers = 23
k23 = KMeans_implementation(n_clusters=centers)
k23.fit(clustering_data_scaleddf)
centroids23=k23.centroids
clusters23=_create_clusters(k23,centroids23,testing_data_scaled)

 19%|█▉        | 58/300 [16:54<1:10:34, 17.50s/it]


In [ ]:
k23c=classify(clusters23)
evaluate(k23c)

label of cluster number  1-> neptune.
within cluster measures:
purity= 0.9854082806210466
recall= 0.6742573283494
f1 score= 0.8006658100689173
----------------
label of cluster number  2-> normal.
within cluster measures:
purity= 1.0
recall= 0.26708826414543024
f1 score= 0.4215779937407346
----------------
label of cluster number  3-> normal.
within cluster measures:
purity= 0.9094258520110531
recall= 0.06182038277711686
f1 score= 0.11577095954660933
----------------
label of cluster number  4-> neptune.
within cluster measures:
purity= 1.0
recall= 0.012345071807987409
f1 score= 0.024389058932128456
----------------
label of cluster number  5-> normal.
within cluster measures:
purity= 1.0
recall= 0.13891845636883518
f1 score= 0.24394802910077149
----------------
label of cluster number  6-> normal.
within cluster measures:
purity= 0.9966688874083944
recall= 0.03122325882328387
f1 score= 0.06054964180191848
----------------
label of cluster number  7-> neptune.
within cluster measures:


### **K=31**

In [ ]:
centers = 31
k31 = KMeans_implementation(n_clusters=centers)
k31.fit(clustering_data_scaleddf)
centroids31=k31.centroids
clusters31=_create_clusters(k31,centroids31,testing_data_scaled)

 22%|██▏       | 67/300 [18:59<1:06:03, 17.01s/it]


In [ ]:
k31c=classify(clusters31)
evaluate(k31c)

label of cluster number  1-> neptune.
within cluster measures:
purity= 1.0
recall= 0.012246704701947669
f1 score= 0.02419707497206161
----------------
label of cluster number  2-> normal.
within cluster measures:
purity= 1.0
recall= 0.1242042869367395
f1 score= 0.2209639091044111
----------------
label of cluster number  3-> satan.
within cluster measures:
purity= 0.7014613778705637
recall= 0.39069767441860465
f1 score= 0.5018670649738611
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 0.999128160418483
recall= 0.023918352013023607
f1 score= 0.04671830411740727
----------------
label of cluster number  5-> normal.
within cluster measures:
purity= 0.8720119521912351
recall= 0.03654540521361634
f1 score= 0.07015083832455278
----------------
label of cluster number  6-> portsweep.
within cluster measures:
purity= 0.5984251968503937
recall= 0.8735632183908046
f1 score= 0.7102803738317758
----------------
label of cluster number  7-> satan.
within clus

### **K=45**

In [ ]:
centers = 45
k45 = KMeans_implementation(n_clusters=centers)
k45.fit(clustering_data_scaleddf)
centroids45=k45.centroids
clusters45=_create_clusters(k45,centroids45,testing_data_scaled)

 22%|██▏       | 66/300 [17:31<1:02:06, 15.93s/it]


In [ ]:
k45c=classify(clusters45)
evaluate(k45c)

label of cluster number  1-> normal.
within cluster measures:
purity= 0.8571428571428571
recall= 0.0006261348694508798
f1 score= 0.0012513556352715444
----------------
label of cluster number  2-> neptune.
within cluster measures:
purity= 1.0
recall= 0.010672831005311825
f1 score= 0.021120249160543093
----------------
label of cluster number  3-> normal.
within cluster measures:
purity= 0.9407069555302167
recall= 0.017218708909899193
f1 score= 0.03381840541094487
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 1.0
recall= 2.0871162315029324e-05
f1 score= 4.1741453437408686e-05
----------------
label of cluster number  5-> normal.
within cluster measures:
purity= 0.6243654822335025
recall= 0.002567152964748607
f1 score= 0.0051132820619413846
----------------
label of cluster number  6-> nmap.
within cluster measures:
purity= 0.4968944099378882
recall= 1.0
f1 score= 0.6639004149377593
----------------
label of cluster number  7-> neptune.
within clu

### **Normalized cut**

In [ ]:
dfn=df3.copy()

In [ ]:
# separate the target column
target_col = dfn.pop('target')



In [ ]:
# create a new dataframe for target column
target_df = pd.DataFrame(target_col)

In [ ]:
target_df.head()

,target
0,normal.
1,normal.
2,normal.
3,normal.
4,normal.


In [ ]:
dfn

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,24,9,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
1,0,1,24,9,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,1,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0
2,0,1,24,9,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,2,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0
3,0,1,24,9,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,3,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0
4,0,1,24,9,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.00,4,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,1,24,9,212,2288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,32,0.0,0.0,0.0,0.0,1.0,0.0,0.16,3,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0
4898427,0,1,24,9,219,236,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,33,0.0,0.0,0.0,0.0,1.0,0.0,0.15,4,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0
4898428,0,1,24,9,218,3610,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,34,0.0,0.0,0.0,0.0,1.0,0.0,0.15,5,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0
4898429,0,1,24,9,219,1234,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,35,0.0,0.0,0.0,0.0,1.0,0.0,0.14,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0


In [ ]:
# create the scaler and fit it to the data
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(dfn.values), columns=dfn.columns, index=dfn.index)

In [ ]:
df_scaled.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,0.5,0.347826,0.9,1.558012e-07,3.441080e-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0.0,0.5,0.347826,0.9,1.173944e-07,3.456654e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.003922,0.003922,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0
2,0.0,0.5,0.347826,0.9,1.710190e-07,9.374494e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.007843,0.007843,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0
3,0.0,0.5,0.347826,0.9,1.688450e-07,1.551219e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.011765,0.011765,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0
4,0.0,0.5,0.347826,0.9,1.731929e-07,3.710101e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005871,0.005871,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.015686,0.015686,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_scaled, target_df,stratify =target_df,train_size=0.005, random_state=42)

calculating degree matrix

In [ ]:
X_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
596302,0.0,0.5,0.550725,0.5,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.401174,0.001957,1.00,1.0,0.00,0.0,0.00,0.07,0.00,1.000000,0.003922,0.00,0.08,0.00,0.00,1.0,1.0,0.00,0.0
337574,0.0,0.5,0.347826,0.9,2.166723e-07,2.984875e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021526,0.046967,0.00,0.0,0.00,0.0,1.00,0.00,0.08,0.764706,1.000000,1.00,0.00,0.01,0.01,0.0,0.0,0.00,0.0
3432304,0.0,1.0,0.710145,0.9,2.029039e-08,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193738,0.189824,0.01,0.0,0.01,0.0,0.98,0.03,0.00,1.000000,0.380392,0.38,0.02,0.38,0.00,0.0,0.0,0.46,0.0
239420,0.0,0.5,0.347826,0.9,2.311655e-07,4.160504e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066536,0.080235,0.00,0.0,0.00,0.0,1.00,0.00,0.12,1.000000,1.000000,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0
870042,0.0,0.5,0.347826,0.9,1.623231e-07,3.450547e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041096,0.041096,0.00,0.0,0.00,0.0,1.00,0.00,0.00,1.000000,1.000000,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0


In [ ]:
X_train_numeric = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)

# Calculate the adjacency matrix
W = cosine_similarity(X_train_numeric)
# degree matrix
D = np.diag(np.sum(W, axis=1))
# Laplacian matrix
L= D - W 
# compute the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(L)

eigenvectors = eigenvectors[:, 0:23]
normalized_eig=LA.norm(eigenvectors) 

kmeans 

In [ ]:
eigenvectors=eigenvectors/normalized_eig

In [ ]:
eigenvectors #use in k means


array([[-2.84437761e-03,  1.09060318e-05,  5.96135113e-05, ...,
        -1.51048376e-04, -5.26476264e-05,  1.54239302e-04],
       [-2.84437761e-03, -1.00882367e-04, -1.70865860e-04, ...,
        -6.31711591e-05,  5.76812177e-05, -1.01304608e-04],
       [-2.84437761e-03, -5.55519701e-05, -8.62426251e-05, ...,
        -2.20140225e-05,  2.27072700e-05, -5.10231383e-05],
       ...,
       [-2.84437761e-03, -1.26492022e-04, -2.14274780e-04, ...,
        -7.86156787e-05,  6.95995012e-05, -1.31533689e-04],
       [-2.84437761e-03, -9.76626856e-05, -1.63351962e-04, ...,
        -6.01748383e-05,  5.14344294e-05, -9.26185332e-05],
       [-2.84437761e-03, -9.46481979e-05, -1.60247790e-04, ...,
        -5.98940037e-05,  5.42081732e-05, -9.39558267e-05]])

In [ ]:
ydf=pd.DataFrame(eigenvectors)
y_train=y_train.reset_index()
y_train.drop(columns="index",inplace=True)
y_train=y_train['target']
ydf['target']=y_train
ydflabel=ydf['target']
ydf.drop(columns="target",inplace=True)

In [ ]:
eigenvectors_df = pd.DataFrame(eigenvectors)

In [ ]:
centers = 23
kN23 = KMeans_implementation(n_clusters=centers)
kN23.fit(ydf)
centroidsN23=kN23.centroids
clustersN23=_create_clusters(kN23,centroidsN23,ydf.to_numpy())

 15%|█▍        | 44/300 [00:28<02:44,  1.56it/s]


In [ ]:
def get_conditional_entropy_per_cluster(lst):
  entropy=0
  # print(lst)
  # print(f"sum is: {lst.sum()}")
  for i in lst:
    if i==0:
      continue
    else:
      ent=(-i/lst.sum())*math.log(i/lst.sum(),2)
      # print(ent)
      entropy+=ent
  return entropy

In [ ]:
def evaluateN(cluster):
  occur=ydflabel.value_counts()
  occurdf=occur.to_frame().transpose()
  col=cluster.columns.to_numpy()
  # convert k7 to numpy array to get indexes of each maximum element.
  clusarr=cluster.to_numpy()
  f1=0
  e=0
  p=0
  r=0

  for cluster, classcount in enumerate(clusarr): #get measures for each cluster 
    maxindex=argmax(classcount) # maximum index number
    maxnumber=classcount[maxindex]
    class_count=0
    clus_arr=0
    clus_arr=clusarr.sum()
    class_count=classcount.sum()
    if maxnumber==0: #maximum element in cluster is 0 so no elements in this cluster 
      print(f"cluster {cluster+1} is empty")
      print(f"----------------")
      continue
    maxclass=col[maxindex] #maximum class column name 
    print(f"label of cluster number  {cluster+1}-> {maxclass}")
    #calculate the purity
    purity=maxnumber/classcount.sum()
    #calculate recall
    recall=maxnumber/occurdf[maxclass][0]
    f1score=(2*purity*recall)/(purity + recall)
    entro_per_cluster=get_conditional_entropy_per_cluster(classcount)
    print("within cluster measures:")
    print(f"purity= {purity}")
    print(f"recall= {recall}") 
    print(f"f1 score= {f1score}")
    # print(f"entropy= {entro}")
    print(f"----------------")
    ratio=class_count/clus_arr
    p+=purity*(ratio)
    r+=recall*(ratio)
    f1+=f1score*(ratio)
    e+=entro_per_cluster*(ratio)
  f1=(2*p*r)/(p+r)
  print("External evaluation:")
  print(f"total purity= {p}")
  print(f"total recall= {r}")
  print(f"total f1 score= {f1}")
  print(f"conditional entropy= {e}")

In [ ]:
def classifyN(clusters):
  clusterdf = pd.DataFrame(columns =ydflabel.unique())
  clusterdf = pd.DataFrame(0, index=np.arange(len(clusters)), columns=ydflabel.unique())
  for cluster_idx, cluster in enumerate(clusters):
    for sample_idx in cluster:
      lab=ydflabel.iloc[sample_idx]
      clusterdf.at[cluster_idx,lab]=clusterdf.at[cluster_idx,lab]+1
  return clusterdf

In [ ]:
ydflabel.unique()

array(['neptune.', 'normal.', 'teardrop.', 'ipsweep.', 'back.', 'satan.',
       'smurf.', 'portsweep.', 'pod.', 'nmap.', 'warezclient.'],
      dtype=object)

In [ ]:
kN23c=classifyN(clustersN23)

In [ ]:
evaluate(kN23c)

label of cluster number  1-> neptune.
within cluster measures:
purity= 0.9833024118738405
recall= 0.8753096614368291
f1 score= 0.9261686325906511
----------------
label of cluster number  2-> neptune.
within cluster measures:
purity= 0.75
recall= 0.002477291494632535
f1 score= 0.0049382716049382715
----------------
label of cluster number  3-> normal.
within cluster measures:
purity= 1.0
recall= 0.08072852571991139
f1 score= 0.1493964928262355
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 1.0
recall= 0.08072852571991139
f1 score= 0.1493964928262355
----------------
label of cluster number  5-> normal.
within cluster measures:
purity= 0.9952015355086372
recall= 0.25523012552301255
f1 score= 0.4062683643486778
----------------
label of cluster number  6-> satan.
within cluster measures:
purity= 0.6111111111111112
recall= 0.44
f1 score= 0.5116279069767442
----------------
label of cluster number  7-> ipsweep.
within cluster measures:
purity= 0.8095

### **other spectral algorithms (DBSCAN)**

In [ ]:
import numpy

def dbscan(D, eps, MinPts):
    
    labels = [0]*len(D) 
    C = 0 # the ID of the current cluster.    
    
    # This outer loop is just responsible for picking new seed points--a point
    # from which to grow a new cluster.
    # Once a valid seed point is found, a new cluster is created, and the 
    # cluster growth is all handled by the 'expandCluster' routine.
    
    # For each point P in the Dataset D...
    # ('P' is the index of the datapoint, rather than the datapoint itself.)
    for P in range(0, len(D)):
    
        # Only points that have not already been claimed can be picked as new 
        # seed points.    
        # If the point's label is not 0, continue to the next point(already chosen).
        if not (labels[P] == 0):
           continue
        
        # Find all of P's neighboring points.
        NeighborPts = region_query(D, P, eps)
        
        # If the number is below MinPts, this point is noise. 
        # This is the only condition under which a point is labeled 
        # NOISE--when it's not a valid seed point. A NOISE point may later 
        # be picked up by another cluster as a boundary point (this is the only
        # condition under which a cluster label can change--from NOISE to 
        # something else).
        if len(NeighborPts) < MinPts:
            labels[P] = -1 #(will be added to the cluster at the end (outliers))
        # Otherwise, if there are at least MinPts nearby, use this point as the 
        # seed for a new cluster.    
        else: 
           C += 1
           grow_cluster(D, labels, P, NeighborPts, C, eps, MinPts)
    
    # All data has been clustered!
    return labels


def grow_cluster(D, labels, P, NeighborPts, C, eps, MinPts):
    '''
    Grow a new cluster with label `C` from the seed point `P`.
    
    This function searches through the dataset to find all points that belong
    to this new cluster. When this function returns, cluster `C` is complete.
    
    Parameters:
      `D`      - The dataset (a list of vectors)
      `labels` - List storing the cluster labels for all dataset points
      `P`      - Index of the seed point for this new cluster
      `NeighborPts` - All of the neighbors of `P`
      `C`      - The label for this new cluster.  
      `eps`    - Threshold distance
      `MinPts` - Minimum required number of neighbors
    '''

    labels[P] = C
    

    i = 0
    while i < len(NeighborPts):    
        
        # Get the next point from the queue.        
        Pn = NeighborPts[i]
      
        if labels[Pn] == -1:
           labels[Pn] = C #outliers
        
        # Otherwise, if Pn isn't already claimed, claim it as part of C.
        elif labels[Pn] == 0:
            # Add Pn to cluster C (Assign cluster label C).
            labels[Pn] = C
            
            # Find all the neighbors of Pn
            PnNeighborPts = region_query(D, Pn, eps)
            
            # If Pn has at least MinPts neighbors, it's a branch point!
            # Add all of its neighbors to the FIFO queue to be searched. 
            if len(PnNeighborPts) >= MinPts:
                NeighborPts = NeighborPts + PnNeighborPts
            # If Pn *doesn't* have enough neighbors, then it's a leaf point.
            # Don't queue up it's neighbors as expansion points.
            #else:
                # Do nothing                
                #NeighborPts = NeighborPts               
        
        # Advance to the next point in the FIFO queue.
        i += 1        
    
    # We've finished growing cluster C!


def region_query(D, P, eps):
    '''
    Find all points in dataset `D` within distance `eps` of point `P`.
    
    This function calculates the distance between a point P and every other 
    point in the dataset, and then returns only those points which are within a
    threshold distance `eps`.
    '''
    neighbors = []
    
    # For each point in the dataset...
    for Pn in range(0, len(D)):
        
        # If the distance is below the threshold, add it to the neighbors list.
        if numpy.linalg.norm(D[P] - D[Pn]) < eps:
           neighbors.append(Pn)
            
    return neighbors

In [ ]:
my_labels = dbscan(X_train.to_numpy(), eps=0.3, MinPts=10)

In [ ]:
def create_cluster_DBSCAN(n_clusters,labels):
    # assign the samples to the closest centroids
    clusters = [[] for _ in range(n_clusters)]
    for i in range(0,len(labels)):
        if labels[i] < 0:
          continue
        clusters[labels[i]].append(i)
    return clusters

In [ ]:
dbscan_classify=classify(create_cluster_DBSCAN(max(my_labels)+1,my_labels))

In [ ]:
evaluate(dbscan_classify)

cluster 1 is empty
----------------
label of cluster number  2-> normal.
within cluster measures:
purity= 0.8998073217726397
recall= 0.22987939945852817
f1 score= 0.36620270535189176
----------------
label of cluster number  3-> normal.
within cluster measures:
purity= 0.8924689440993789
recall= 0.5658380507014521
f1 score= 0.6925741828588643
----------------
label of cluster number  4-> normal.
within cluster measures:
purity= 0.9166666666666666
recall= 0.002707359094265321
f1 score= 0.005398773006134969
----------------
label of cluster number  5-> normal.
within cluster measures:
purity= 0.8909090909090909
recall= 0.036180162441545656
f1 score= 0.06953642384105961
----------------
label of cluster number  6-> normal.
within cluster measures:
purity= 0.8372093023255814
recall= 0.008860447944868325
f1 score= 0.01753531417437896
----------------
label of cluster number  7-> normal.
within cluster measures:
purity= 0.898936170212766
recall= 0.0415948806300763
f1 score= 0.079510703363914